In [26]:
%reset -f

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import tools
import time 

In [27]:
# Parameters to tweak
lr = 1e-3
batchSize = 64
epochs = 2
loss_fn = nn.CrossEntropyLoss()

In [28]:
training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

# Create 64 batches
train_dataloader = DataLoader(training_data, batch_size=batchSize)
test_dataloader = DataLoader(test_data, batch_size=batchSize)
len(train_dataloader)

938

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [30]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [31]:
def train_data(model, optimizer):
    for xb, yb in train_dataloader:
        preds = model(xb)
        loss = loss_fn(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss = loss.item()
    print(f"Train loss: {loss:>7f}")

In [32]:
def test_data(model):
    num_batches = len(test_dataloader)
    size = len(test_dataloader.dataset)
    test_loss, corrects = 0, 0

# test_results = [(np.argmax(self.feedforward(x)), y)
#                        for (x, y) in test_data]

    with torch.no_grad():
        for xb, yb in test_dataloader:
            preds = model(xb)
            test_loss += loss_fn(preds, yb).item()
            corrects += (preds.argmax(1) == yb).type(torch.float).sum().item()

    test_loss /= num_batches
    # test_loss = lo
    corrects /= size

    print(f"Test loss: \n Accuracy: {(100*corrects):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return corrects

In [33]:
model = NeuralNetwork().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
epoch_pro = []
epoch_performance = []
for t in range(epochs):
    start_time = time.time()

    train_data(model, optimizer)
    epoch_pro.append(test_data(model))
    epoch_performance.append(time.time()-start_time)
        

Train loss: 2.149687
Test loss: 
 Accuracy: 33.6%, Avg loss: 2.158493 

Train loss: 1.936472
Test loss: 
 Accuracy: 61.3%, Avg loss: 1.923246 



In [34]:
tools.saveResult(epochs,lr,"crossentropyloss",batchSize,epoch_pro,epoch_performance, "model_cross_entropy_sigmoid")

In [35]:
epoch_pro

[0.3355, 0.6131]